#### Global & Local Modules

In [1]:
import os
from itertools import permutations 
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from rgcn import GaussianGraphConvolution, kl_reg

Using TensorFlow backend.


#### Vendor Modules

In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS
# import vendor modules
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits

# Data

In [3]:
def attack_edges(attack_ratio: float) -> 'callable':
    """
    Create an edge attack function with given attack ratio.

    Args:
        attack_ratio: the ratio of noise to clean edges

    Returns:
        a callable function for attacking a graph

    """
    def _attack_edges(edges: np.ndarray) -> np.ndarray:
        f"""
        Attack the edges of the given list of edges.

        Args:
            edges: the ndarray of tuples representing edges to attack

        Returns:
            a new ndarray of edges with random edges added

        Note:
            the ratio of noise to clean edges is {attack_ratio}

        """
        # create the set of all possible edges
        all_edges = set(permutations(list(range(edges.max())), 2))
        # convert the ndarray of edges to a set of tuples
        edges = set(map(tuple, edges))
        # create the set of edges to sample by subtracting the existing
        # edges from the set of possible edges
        sample_edges = all_edges - edges
        # select random edges without replacement based on the noise ratio
        attack_edges = np.random.choice(list(range(len(sample_edges))), 
            size=int(len(edges) * attack_ratio), 
            replace=False)
        attack_edges = [edge for (i, edge) in enumerate(list(sample_edges)) if i in attack_edges]
        # create the new ndarray of edges with the attack edges added
        edges = list(edges) + attack_edges
        return np.array(edges)
    return _attack_edges

In [4]:
X, A, y = load_data('data/cora/', dataset='cora', attack_edges=attack_edges(0))
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask, val_mask, test_mask = get_splits(y)

In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

# Model

## Input Layers

In [8]:
X_in = Input(shape=(X.shape[1], ), name='features')
X_in

<tf.Tensor 'features:0' shape=(?, 1433) dtype=float32>

In [9]:
A_in = Input(shape=(None, None), batch_shape=(None, None), sparse=True, name='graph')
A_in

## Training Loop

In [10]:
def train(model, epochs=200, patience=10):
    """
    Train the given model.

    Args:
        model: the model to train
        epochs: the maximum number of training epochs
        patience: the number of patience epochs for early stoping

    Returns:
        a tuple of the testing loss and accuracy

    """
    history = model.fit([X, A], y_train,
        sample_weight=train_mask,
        batch_size=A.shape[0],
        epochs=epochs,
        shuffle=False,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=patience),
        ],
        validation_data = ([X, A], y_val, val_mask)
    )
    val = model.evaluate([X, A], y_test, 
        sample_weight=test_mask, 
        batch_size=A.shape[0],
    )
    return val

## GCN (baseline)

In [11]:
# create the layers
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, 1, 
    activation='relu', 
    kernel_regularizer=l2(5e-4)
)([H, A_in])
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], 1, 
    activation='softmax'
)([H, A_in])
# create the model
gcn = Model(inputs=[X_in, A_in], outputs=Y)
gcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
gcn.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]         

In [12]:
train(gcn)[1]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 0s 95us/step - loss: 1.9766 - weighted_acc: 0.1357 - val_loss: 1.9597 - val_weighted_acc: 0.3567
Epoch 2/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9594 - weighted_acc: 0.3071 - val_loss: 1.9414 - val_weighted_acc: 0.3567
Epoch 3/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9383 - weighted_acc: 0.3571 - val_loss: 1.9225 - val_weighted_acc: 0.3533
Epoch 4/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9202 - weighted_acc: 0.3429 - val_loss: 1.9041 - val_weighted_acc: 0.3600
Epoch 5/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.8999 - weighted_acc: 0.3571 - val_loss: 1.8872 - val_weighted_acc: 0.3633
Epoch 6/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.8791 - w

Epoch 51/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.3018 - weighted_acc: 0.7571 - val_loss: 1.4698 - val_weighted_acc: 0.6933
Epoch 52/200
2708/2708 [==============================] - 0s 16us/step - loss: 1.2628 - weighted_acc: 0.7857 - val_loss: 1.4619 - val_weighted_acc: 0.7100
Epoch 53/200
2708/2708 [==============================] - 0s 22us/step - loss: 1.2552 - weighted_acc: 0.7714 - val_loss: 1.4531 - val_weighted_acc: 0.7167
Epoch 54/200
2708/2708 [==============================] - 0s 23us/step - loss: 1.2588 - weighted_acc: 0.7500 - val_loss: 1.4441 - val_weighted_acc: 0.7233
Epoch 55/200
2708/2708 [==============================] - 0s 26us/step - loss: 1.2315 - weighted_acc: 0.7714 - val_loss: 1.4354 - val_weighted_acc: 0.7233
Epoch 56/200
2708/2708 [==============================] - 0s 21us/step - loss: 1.2759 - weighted_acc: 0.7786 - val_loss: 1.4270 - val_weighted_acc: 0.7233
Epoch 57/200
2708/2708 [==============================] - 0s 21us/step

Epoch 104/200
2708/2708 [==============================] - 0s 16us/step - loss: 0.9108 - weighted_acc: 0.8857 - val_loss: 1.1617 - val_weighted_acc: 0.8000
Epoch 105/200
2708/2708 [==============================] - 0s 20us/step - loss: 0.9076 - weighted_acc: 0.8786 - val_loss: 1.1565 - val_weighted_acc: 0.8000
Epoch 106/200
2708/2708 [==============================] - 0s 19us/step - loss: 0.8904 - weighted_acc: 0.8857 - val_loss: 1.1526 - val_weighted_acc: 0.7933
Epoch 107/200
2708/2708 [==============================] - 0s 23us/step - loss: 0.8975 - weighted_acc: 0.9143 - val_loss: 1.1494 - val_weighted_acc: 0.7933
Epoch 108/200
2708/2708 [==============================] - 0s 16us/step - loss: 0.8903 - weighted_acc: 0.9000 - val_loss: 1.1461 - val_weighted_acc: 0.7933
Epoch 109/200
2708/2708 [==============================] - 0s 19us/step - loss: 0.9290 - weighted_acc: 0.8643 - val_loss: 1.1430 - val_weighted_acc: 0.7967
Epoch 110/200
2708/2708 [==============================] - 0s 18

Epoch 157/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.7480 - weighted_acc: 0.9571 - val_loss: 1.0397 - val_weighted_acc: 0.8133
Epoch 158/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.7159 - weighted_acc: 0.9500 - val_loss: 1.0360 - val_weighted_acc: 0.8133
Epoch 159/200
2708/2708 [==============================] - 0s 22us/step - loss: 0.7363 - weighted_acc: 0.9571 - val_loss: 1.0333 - val_weighted_acc: 0.8133
Epoch 160/200
2708/2708 [==============================] - 0s 18us/step - loss: 0.7293 - weighted_acc: 0.9429 - val_loss: 1.0305 - val_weighted_acc: 0.8100
Epoch 161/200
2708/2708 [==============================] - 0s 17us/step - loss: 0.7124 - weighted_acc: 0.9357 - val_loss: 1.0275 - val_weighted_acc: 0.8133
Epoch 162/200
2708/2708 [==============================] - 0s 21us/step - loss: 0.7153 - weighted_acc: 0.9357 - val_loss: 1.0262 - val_weighted_acc: 0.8167
Epoch 163/200
2708/2708 [==============================] - 0s 17

0.8289991617202759

## Gaussian Graph Convolution

In [13]:
# create the layers
H = Dropout(0.6)(X_in)
M1, S1 = GaussianGraphConvolution(16,
    is_first=True,
    dropout=0.6
)([H, A_in])
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    last_activation='softmax',
)([M1, S1, A_in])
# create the model
rgcn = Model(inputs=[X_in, A_in], outputs=Y)
rgcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), weighted_metrics=['acc'])
rgcn.add_loss(kl_reg(M1, S1, 5e-4), [M1, S1])
rgcn.summary()

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                        

In [14]:
train(rgcn)[1]

Train on 2708 samples, validate on 2708 samples
Epoch 1/200
2708/2708 [==============================] - 0s 115us/step - loss: 1.9896 - weighted_acc: 0.1071 - val_loss: 1.9695 - val_weighted_acc: 0.1367
Epoch 2/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9675 - weighted_acc: 0.2071 - val_loss: 1.9560 - val_weighted_acc: 0.2567
Epoch 3/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.9552 - weighted_acc: 0.2786 - val_loss: 1.9499 - val_weighted_acc: 0.2967
Epoch 4/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.9497 - weighted_acc: 0.2929 - val_loss: 1.9408 - val_weighted_acc: 0.3300
Epoch 5/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.9317 - weighted_acc: 0.4429 - val_loss: 1.9316 - val_weighted_acc: 0.4300
Epoch 6/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.9211 - weighted_acc: 0.4929 - val_loss: 1.9228 - val_weighted_acc: 0.4367
Epoch 7/200
2708/2708 [====

Epoch 54/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.4411 - weighted_acc: 0.8071 - val_loss: 1.5980 - val_weighted_acc: 0.7400
Epoch 55/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.4246 - weighted_acc: 0.8143 - val_loss: 1.5925 - val_weighted_acc: 0.7367
Epoch 56/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.4179 - weighted_acc: 0.7929 - val_loss: 1.5928 - val_weighted_acc: 0.7533
Epoch 57/200
2708/2708 [==============================] - 0s 17us/step - loss: 1.4150 - weighted_acc: 0.7929 - val_loss: 1.5952 - val_weighted_acc: 0.7500
Epoch 58/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.4023 - weighted_acc: 0.8000 - val_loss: 1.5729 - val_weighted_acc: 0.7600
Epoch 59/200
2708/2708 [==============================] - 0s 21us/step - loss: 1.4060 - weighted_acc: 0.8000 - val_loss: 1.5841 - val_weighted_acc: 0.7567
Epoch 60/200
2708/2708 [==============================] - 0s 16us/step

Epoch 107/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.1567 - weighted_acc: 0.8786 - val_loss: 1.4034 - val_weighted_acc: 0.7867
Epoch 108/200
2708/2708 [==============================] - 0s 21us/step - loss: 1.1245 - weighted_acc: 0.9143 - val_loss: 1.4012 - val_weighted_acc: 0.7900
Epoch 109/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.1083 - weighted_acc: 0.8929 - val_loss: 1.3754 - val_weighted_acc: 0.8200
Epoch 110/200
2708/2708 [==============================] - 0s 19us/step - loss: 1.1446 - weighted_acc: 0.8786 - val_loss: 1.3821 - val_weighted_acc: 0.7933
Epoch 111/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.1715 - weighted_acc: 0.9000 - val_loss: 1.3571 - val_weighted_acc: 0.8100
Epoch 112/200
2708/2708 [==============================] - 0s 18us/step - loss: 1.1219 - weighted_acc: 0.9143 - val_loss: 1.3957 - val_weighted_acc: 0.7967
Epoch 113/200
2708/2708 [==============================] - 0s 18

0.7899990081787109